Week 3 Segmenting and Clustering Neighborhoods in Toronto

First, we import the relevant libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from bs4 import BeautifulSoup
import csv

Scrape the relevant information from the website.

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).content
content = BeautifulSoup(requests.get(url).content, 'lxml')

To create the required dataframe:

-The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

-Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

-More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia  page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

-If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

-Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

-In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [3]:
# Webscrape the data
table = content.find('table')
td = table.find_all('td')
postcode = []
borough = []
neighbourhood = []

# create a list with the scraped data
for i in range(0, len(td), 3):
    postcode.append(td[i].text.strip())
    borough.append(td[i+1].text.strip())
    neighbourhood.append(td[i+2].text.strip())

In [4]:
# create the actual DataFrame with the lists previously scraped and give the columns appropriate names  
df_codes = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_codes.columns = ['Postal Code', 'Borough', 'Neighborhood']

The next step requires us to follow some of the project rules requirements. In this case, it would be ignoring boroughs with the 'Not assigned' value. Also, if a cell has a borough but a 'Not assigned' neighborhood value, then the neighborhood value will be the same as the borough value for that particular row.

In [5]:
# Ignore cells with a borough that is Not assigned.
df_codes['Borough'].replace('Not assigned', np.nan, inplace=True)
df_codes.dropna(subset=['Borough'], inplace=True)

# Also, if a cell has a borough but a 'Not assigned' neighborhood value, 
# then the neighborhood value will be the same as the borough value for that particular row.
df_codes['Neighborhood'].replace('Not assigned', "Queen's Park", inplace=True)

Next we combine neighborhoods if they exist in one postal code.

In [6]:
# combining the neighborhoods into one line separated by a comma.
df_codes = df_codes.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_codes.columns = ['Postal Code', 'Borough', 'Neighborhood']
df_codes.head(12)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [7]:
#last step is to use the .shape function
df_codes.shape

(103, 3)